In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            ;;[clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [5]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative path to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### New way of using order

In [6]:
(init-portfolio "1980-12-15" 100000)

(order "AAPL" 10) ;with leverage, exact value trade
(order "AAPL" 10 :leverage false) ;without leverage, exact value trade
(order "AAPL" 10 :remaining true) ;with leverage, remaining value
(order "AAPL" 10 :leverage false :remaining true) ;without leverage, remaining value

nil

In [7]:
(pprint/print-table (deref order-record))


|      :date | :tic |  :price | :aprc | :quantity |
|------------+------+---------+-------+-----------|
| 1980-12-16 | AAPL | 25.3125 | 25.82 |        10 |
| 1980-12-16 | AAPL | 25.3125 | 25.82 |        10 |
| 1980-12-16 | AAPL | 25.3125 | 25.82 |       -10 |


nil